**User input categories**
* Product
* Return reason
* Brand
* Timeline
* Claimed item condition

In [1]:
# For this demonstration, I will go ahead and make up answers for each of the user input categories to show proof-of-concept

product  = "topwear"
return_reason = "defective"
brand = "uniqlo"
timeline = "within 30 days"
item_condition = "gently used"

Allow the user to upload multiple images. 

Prompt engineer Gemini to return 1 of 3 final categories: **Return to store, donate, recycle**

In [2]:
from IPython.display import display
from ipywidgets import FileUpload
from PIL import Image
import io

upload_widget = FileUpload(accept='image/*', multiple=True)
display(upload_widget)

uploaded_images = {}

def handle_upload(change):
    global uploaded_images
    uploaded_images.clear()  # Reset previous uploads
    for fileinfo in upload_widget.value:
        filename = fileinfo["name"]
        image_data = fileinfo["content"]
        image = Image.open(io.BytesIO(image_data))
        uploaded_images[filename] = image
        display(image)

upload_widget.observe(handle_upload, names='value')


FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

In [10]:
from google import genai
from google.genai import types
import io
import os
from dotenv import load_dotenv

load_dotenv()
gemini_key = os.getenv("GEMINI_API_KEY")

if not uploaded_images:
    raise ValueError("No images uploaded yet. Please upload images first.")

# Convert the images into the correct format (types.Part)
gemini_images = []

for filename, pil_image in uploaded_images.items():
    # If the image has an alpha channel (RGBA), convert it to RGB
    if pil_image.mode == "RGBA":
        pil_image = pil_image.convert("RGB")
    
    img_bytes_io = io.BytesIO()
    
    # Convert the PIL Image to a byte stream (JPEG or PNG)
    pil_image.save(img_bytes_io, format="JPEG")  
    img_bytes_io.seek(0)  
    
    # Convert to types.Part for Gemini
    gemini_images.append(
        types.Part.from_bytes(
            data=img_bytes_io.getvalue(),  # Get the byte data from the BytesIO object
            mime_type="image/jpeg"  # Use "image/png" if your images are in PNG format
        )
    )

client = genai.Client(api_key=gemini_key)
response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=[f"""
    Your job is to assess the product condition based on the images provided into 3 categories: 1. Return to store; 2: Recycle; 3: Donate. The customer wants to return a {product} from {brand} because it is {return_reason}. 
    The item is {item_condition} and they purchased it {timeline}. Categorize them and if the images are ambiguous, say so.
    """] + gemini_images
)

# the issue here is that we haven't defined (ourselves) what the criteria is for return to store, recycle, donate

print(response.text)

response_numeric = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=[f"""
    Your job is to assess the product condition based on the images provided into 3 categories: 1. Return to store; 2: Recycle; 3: Donate. The customer wants to return a {product} from {brand} because it is {return_reason}. 
    The item is {item_condition} and they purchased it {timeline}. ONLY output the number for the category. If it is ambiguous, output 4.
    """] + gemini_images
)

print(response_numeric.text)

Based on the image provided, here is the assessment of the product:

*   **Return to Store:**

    The shirt appears heavily creased/wrinkled throughout the front and back. While it's hard to confirm the defect without seeing it, the overall condition suggests the shirt is not in resalable condition. Given that it was purchased within 30 days and has been gently used, the customer is eligible for a refund if they have proof of purchase.
1

